In [1]:
import random

def generateBoard(cols, rows, mines):
    board = [[0 for x in range(cols)] for y in range(rows)]

    #generate mine locations
    minePos = random.sample(range(rows * cols), mines)
    for pos in minePos:
        row, col = divmod(pos, cols)
        board[row][col] = 'M'

    #count neighbors
    for row in range(rows):
        for col in range(cols):
            if board[row][col] == 'M':
                continue
            count = 0
            for rowShift in [-1, 0, 1]:
                for colShift in [-1, 0, 1]:
                    r= row + rowShift
                    c = col + colShift
                    if (0 <= r < rows) and (0 <= c < cols) and (board[r][c] == 'M'):
                        count += 1
            board[row][col] = count

    return board

def generatePlayArea(cols, rows):
    PlayArea = [['X' for x in range(cols)] for y in range(rows)]
    return PlayArea

def reveal_cell(board, play_area, col, row):
    if play_area[row][col] != 'X':
        return False
    if board[row][col] == 'M':
        play_area[row][col] = 'M'
        return True
    elif board[row][col] > 0:
        play_area[row][col] = str(board[row][col])
    else:
        play_area[row][col] = ' '
        for row_shift in [-1, 0, 1]:
            for col_shift in [-1, 0, 1]:
                r = row + row_shift
                c = col + col_shift
                if 0 <= r < len(board) and 0 <= c < len(board[0]):
                    if play_area[r][c] == 'X':
                        reveal_cell(board, play_area, c, r)
    return False

def print_board(play_area):
    for row in play_area:
        print(" ".join(row))
    print()

def check_win(board, play_area):
    for row in range(len(board)):
        for col in range(len(board[0])):
            if board[row][col] != 'M' and play_area[row][col] == 'X':
                return False
    return True

def checkSpot(board, col, row):
    result = board[col][row]
    return result

In [2]:
import copy

# class used to store the information of a cluster of cells found by the solver
class Cluster():
    def __init__(self, cells, count):
        self.cells = set(cells)
        self.count = count

    def __eq__(self, other):
        return self.cells == other.cells and self.count == other.count

    # used to find out if there are mines known in the cluster
    def known_mines(self):
        if len(self.cells)==self.count:
            return self.cells
        else:
            return set()

    # used to find out if there are safe cells known in the cluster    
    def known_safes(self):
        if self.count==0:
            return self.cells
        else:
            return set()

    # used to mark the mines that are found in the cluster
    def mark_mine(self, cell):
        if cell in self.cells:
            self.cells.discard(cell)
            self.count-=1

    # used to mark the safe cells that are found in the cluster
    def mark_safe(self, cell):
        if cell in self.cells:
            self.cells.discard(cell)

class Solver():
    def __init__(self, board, play_area, cols, rows):
        self.board = board
        self.play_area = play_area
        self.cols = cols
        self.rows = rows

        self.explored = set()

        self.mines = set()
        self.safes = set()
        self.kb = []

        # set of all possible moves in the board; used for comparison later on
        self.legal_moves = set()
        for i in range(rows):
            for j in range(cols):
                self.legal_moves.add((i,j))

    # used to make a move based on the knowledge base
    # if a move can be deemed safe and has not been explored, return it
    # otherwise, return None
    def make_move(self):
        for move in self.safes:
            if move not in self.explored:
                self.explored.add(move)
                return move
        return None

    # used to make a move when no safe move can be made
    # if a move exists that has not been explored and has not been deemed a mine, return it
    # otherwise, return None
    def make_guess(self):
        for i in range(self.rows):
            for j in range(self.cols):
                if((i,j) not in self.explored):
                    if (i,j) not in self.mines:
                        self.explored.add((i,j))
                        return (i,j)
                    else:
                        return "death"
        return None
    
    # used to mark a cell as a mine
    # calls function existing inside the cluster
    def mark_mine(self, cell):
        self.mines.add(cell)
        for cluster in self.kb:
            cluster.mark_mine(cell)

    # used to mark a cell as safe
    # calls function existing inside the cluster
    def mark_safe(self, cell):
        self.safes.add(cell)
        for cluster in self.kb:
            cluster.mark_safe(cell)

    # functions for handling the exceptions dealing with the counts of the clusters
    # makes sure that there are no illegal inputs
    def get_count(self, move, cells):
        if self.play_area[move[0]][move[1]] == ' ':
            return 0
        elif self.board[move[0]][move[1]] == 'M':
            c = 0
            for i, j in cells:
                if self.board[i][j] == 'M':
                    c += 1
            return c
        else:
            return int(self.play_area[move[0]][move[1]])

    def update_kb(self, move):
        # mark the move made as one that is safe since it bypassed checks
        self.explored.add(move)
        self.safes.add(move)

        # get cluster of cells that are adjacent to the move made
        cells={(move[0],move[1]-1),(move[0],move[1]+1),(move[0]+1,move[1]),(move[0]-1,move[1]),(move[0]+1,move[1]+1),(move[0]-1,move[1]-1),(move[0]+1,move[1]-1),(move[0]-1,move[1]+1)}
        # remove cells that are out of bounds
        cells = cells.intersection(self.legal_moves)
        # remove cells that are already passed
        cells = cells.difference(self.explored)
        # add the new cluster to the knowledge base
        self.kb.append(Cluster(cells=cells, count=self.get_count(move, cells)))
        while True:
            # loop through the existing clusters of cells in the knowledge base to find out existing knowledge and try to make new discoveries
            wkb=copy.deepcopy(self.kb)
            for cluster in wkb:
                safe=cluster.known_safes()
                mine=cluster.known_mines()
                if len(safe): # check for new safe cells
                    self.safes.update(safe)
                    self.kb.remove(cluster)
                if len(mine): # check for new mine cells
                    self.mines.update(mine)
                    self.kb.remove(cluster)

            # re-mark mines to make sure there is full coverage based on new info
            for s in self.safes:
                self.mark_safe(s)
            for m in self.mines:
                self.mark_mine(m)

            # find new clusters given the current knowledge base and the discoveries made above
            wkb2 = copy.deepcopy(self.kb)
            for c1 in wkb2:
                for c2 in wkb2:
                    newcells = c2.cells.difference(c1.cells)
                    newcluster = Cluster(cells=newcells,count=(c2.count-c1.count))
                    if not(c2 == c1) and c1.cells.issubset(c2.cells) and (newcluster not in self.kb):
                        self.kb.append(newcluster)
            if(wkb==self.kb):
                break

        # account for if the move was blank and spread to other cells
        if self.play_area[move[0]][move[1]] == ' ':
            for i in range(self.rows):
                for j in range(self.cols):
                    if self.play_area[i][j] != 'X' and (i, j) not in self.explored:
                        self.update_kb((i,j))
    
    def solve(self):
        move = (0,0) # the most optimal starting spot is in the corner, use top left for ease of use
        while (not reveal_cell(self.board, self.play_area, move[1], move[0])) and (not check_win(self.board, self.play_area)):
            # handle the update of the AI model for the next move
            self.update_kb(move)

            # handle choosing the next move for the AI
            # None means that there are no logical steps to take in that route, so when both options return None, the game has been solved
            move = self.make_move()
            if move is None:
                move = self.make_guess()
                if move == "death":
                    return False
                elif move is None:
                    break
            
        # check whether game was won or lost
        if check_win(self.board, self.play_area):
            return True
        else:
            return False

In [3]:
# test class in order to test the accuracy of the Solver class
class TestSolver():
    def __init__(self, cols, rows, mines):
        self.cols = cols
        self.rows = rows
        self.mines = mines

        self.board = generateBoard(cols, rows, mines)
        self.play_area = generatePlayArea(cols, rows)

        self.ai = Solver(self.board, self.play_area, cols, rows)

    def test_solve_same_board(self, iterations):
        results = dict()
        for i in range(iterations):
            result = self.ai.solve()
            if result in results:
                results[result] += 1
            else:
                results[result] = 1

        print(f"Accuracy of results across same board: {results[True]/iterations} ({results[True]} solved boards across {iterations} games)")

    def test_solve_diff_board(self, iterations):
        results = dict()
        for i in range(iterations):
            self.board = generateBoard(self.cols, self.rows, self.mines)
            self.play_area = generatePlayArea(self.cols, self.rows)
            self.ai = Solver(self.board, self.play_area, self.cols, self.rows)

            result = self.ai.solve()
            if result in results:
                results[result] += 1
            else:
                results[result] = 1

        print(f"Accuracy of results across different board: {results[True]/iterations} ({results[True]} solved boards across {iterations} games)")

In [4]:
testSolver = TestSolver(10, 10, 10)
testSolver = TestSolver(10, 10, 10)
testSolver.test_solve_diff_board(20)

Accuracy of results across different board: 0.7 (14 solved boards across 20 games)
